#### Pull next unresolved Alias from DB and Resolve it

In [1]:
import sys
import mariadb
import time
import hashlib
import json
from datetime import datetime as datingdays

local_lib_dir = '../../../python/lib/'
sys.path.append(local_lib_dir)
from kitchen_sink_class import Query
from kitchen_sink_class import load_single_line_from_file as load_it
q = Query()

pwd = load_it('./db.pwd')
mydb = mariadb.connect(
    port=3306,
    host='127.0.0.1',
    user='w3hacknet',
    password=pwd,
    database='w3hacknet',
    autocommit=True
)
cursor = mydb.cursor()
commit_set = None
running = True
call_count = 0
alias_count = 0
fail_count = 0
try_counters = [0, 0, 0]

while running:
#    print(datingdays.now().isoformat(), 'calling ReserveNextUnresolvableAlias')
    cursor.callproc('ReserveNextUnresolvedAlias', ('krunch', None))
    if cursor.sp_outparams:
#        print(datingdays.now().isoformat(), 'returned from ReserveNextUnresolvableAlias')
        alias_count += 1
        result_set = cursor.fetchone()
        if result_set is not None and len(result_set) > 0:
            commit_set = json.loads(result_set[0])
            idx = 0
            alias_id = -1
            alias_not_found = True
            for n in commit_set:
                #Need to load the commit_id->owner/name pair map
                #Originally this was loaded first as start of a much larger process
                q.add_commit_id(n['commit_id'], n['owner'], n['name'])
                alias_id = n['alias_id']
            while alias_not_found and idx < len(commit_set):
                commit_id = commit_set[idx]['commit_id']
                call_count += 1
                if call_count % 10 == 0:
                    print(datingdays.now().isoformat(),'Aliases Processed',alias_count,',Call Count',call_count,',Unresolvable Count',fail_count, try_counters)

    #            print(datingdays.now().isoformat(), 'calling retrieve_commit')

                resp = q.retrieve_commit(commit_id)

    #            print(datingdays.now().isoformat(), 'returned from retrieve_commit')

                if (resp != None):
                    j = resp.json()
                    commit_details_block = j['author']
                    if commit_details_block is not None and 'login' in commit_details_block.keys():                
                        committer = commit_details_block['login']

    #                    print(datingdays.now().isoformat(), 'calling ResolveAliasViaPrimaryKey')
                        cursor.callproc('ResolveAliasViaPrimaryKey', (alias_id, committer))
    #                    print(datingdays.now().isoformat(), 'returned from ResolveAliasViaPrimaryKey')

                        try_counters[idx] += 1
                        alias_not_found = False
                idx += 1
        if alias_not_found:
            fail_count += 1
            cursor.callproc('ResolveAliasViaPrimaryKey', (alias_id, '<UNABLE_TO_RESOLVE>'))
            print('Unable to resolve', alias_id)
    else:
        running = False
        print('Ran out of aliases')

print('done!')
#raise Exception('Do not continue!')

TypeError: the JSON object must be str, bytes or bytearray, not NoneType